In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

C:\Users\Hp\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [23]:
#function to collect the necessary data
def collect_data():
    route_starting_elements = driver.find_element(By.XPATH,"//span[@class='src']")
    route_reaching_elements = driver.find_element(By.XPATH,"//span[@class='dst']")
    names_elements = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    fares_elements = driver.find_elements(By.XPATH, "//div[@class='fare d-block']")
    ratings_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
    seats_elements = driver.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']")
    bus_types_elements = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    starting_time_elements = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    duration_elements = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    ending_time_elements = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    starting_point_elements = driver.find_elements(By.XPATH, "//div[@class='dp-loc l-color w-wrap f-12 m-top-42']")
    
    
    Starting_name = route_starting_elements.text
    Reaching_name =  route_reaching_elements.text
    names = [i.text for i in names_elements]
    fares = [i.text for i in fares_elements]
    ratings = [i.text for i in ratings_elements]
    seats = [i.text for i in seats_elements]
    bus_types = [i.text for i in bus_types_elements]
    starting_time = [i.text for i in starting_time_elements]
    duration = [i.text for i in duration_elements]
    ending_time = [i.text for i in ending_time_elements]
    starting_point = [i.text for i in starting_point_elements]

    max_length = max(len(Reaching_name),len(Starting_name),len(names), len(fares), len(ratings), len(seats), len(bus_types), len(starting_time), len(duration), len(ending_time), len(starting_point))

    def fill_to_max_length(lst, max_length):
        return lst + [np.nan] * (max_length - len(lst))
    
    #starting_names = fill_to_max_length(Starting_names, max_length)
    #Reaching_names = fill_to_max_length(Reaching_names, max_length)
    names = fill_to_max_length(names, max_length)
    fares = fill_to_max_length(fares, max_length)
    ratings = fill_to_max_length(ratings, max_length)
    seats = fill_to_max_length(seats, max_length)
    bus_types = fill_to_max_length(bus_types, max_length)
    starting_time = fill_to_max_length(starting_time, max_length)
    duration = fill_to_max_length(duration, max_length)
    ending_time = fill_to_max_length(ending_time, max_length)
    starting_point = fill_to_max_length(starting_point, max_length)
    
    Starting_name = [Starting_name]*max_length
    Reaching_name = [Reaching_name]*max_length
    
    bus_details = {
        'Starting_names' : Starting_name,
        'Reaching_names' : Reaching_name,
        'Names': names,
        'Bus_fare': fares,
        'Ratings': ratings,
        'Seats': seats,
        'Bus_types': bus_types,
        'Starting_time': starting_time,
        'Duration': duration,
        'Ending_time': ending_time,
        'Starting_point': starting_point
    }

    return bus_details

In [28]:
# Andhra dataset


# Initialize the WebDriver
driver = webdriver.Chrome()
driver.maximize_window()
driver.implicitly_wait(10)
driver.get("https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile")

wait = WebDriverWait(driver, 20)

# Function to collect route links from the current page
def collect_route_links():
    route_details_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
    route_details_links = [element.get_attribute('href') for element in route_details_elements]
    return route_details_links

# Collect route links from the first page
all_route_links = collect_route_links()

# Iterate over pagination pages to collect more route links
page_number = 1
while True:
    try:
        # Locate the pagination container
        pagination_container = wait.until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
        ))

        # Locate the next page button within the container
        next_page_button = pagination_container.find_element(
            By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
        )
        
        # Scroll the next page button into view
        driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
        time.sleep(1)  # Wait for a bit after scrolling
        
        # Log the action
        print(f"Clicking on page {page_number + 1}")
        
        # Click the next page button
        next_page_button.click()
        
        # Wait for the page number to update to the next page
        wait.until(EC.text_to_be_present_in_element(
            (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
        
        # Log the successful page navigation
        print(f"Successfully navigated to page {page_number + 1}")
        
        # Wait for a short duration to ensure the next page loads completely
        time.sleep(3)

        # Collect route links from the current page
        all_route_links.extend(collect_route_links())
        
        page_number += 1

    except Exception as e:
        print(f"No more pages to navigate or an error occurred: {e}")
        break

# Remove duplicates, if any
all_route_links = list(set(all_route_links))

# Display the collected route links
print(f"Collected {len(all_route_links)} route links:")

all_data = []
for link in all_route_links:
        driver.get(link)
        time.sleep(5)  # Adjust this sleep time as needed

        # Scroll to the bottom to load all content
        loading_waiting_time = 1
        previous_page_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(loading_waiting_time)
            new_page_height = driver.execute_script("return document.body.scrollHeight")
            if new_page_height == previous_page_height:
                break
            previous_page_height = new_page_height

        bus_details = collect_data()
        all_data.append(bus_details)

# Close the WebDriver
driver.quit()


Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
Clicking on page 5
Successfully navigated to page 5
No more pages to navigate or an error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7A7FB3E32+31618]
	(No symbol) [0x00007FF7A7F2B099]
	(No symbol) [0x00007FF7A7DE888A]
	(No symbol) [0x00007FF7A7E38524]
	(No symbol) [0x00007FF7A7E3862C]
	(No symbol) [0x00007FF7A7E7F787]
	(No symbol) [0x00007FF7A7E5D14F]
	(No symbol) [0x00007FF7A7E7CA80]
	(No symbol) [0x00007FF7A7E5CEB3]
	(No symbol) [0x00007FF7A7E2A46B]
	(No symbol) [0x00007FF7A7E2B001]
	GetHandleVerifier [0x00007FF7A82B9FFD+3202381]
	GetHandleVerifier [0x00007FF7A8306A1D+3516269]
	GetHandleVerifier [0x00007FF7A82FC490+3473888]
	GetHandleVerifier [0x00007FF7A8065D36+760454]
	(No symbol) [0x00007FF7A7F36B3F]
	(No symbol) [0x00007FF7A7F31CD4]
	(No symbol) [0x00007FF7A7F31E62]
	(No symbol) [0x00007FF7A7F2120F]
	(No symb

In [33]:
andhra_df = pd.DataFrame()

for data in all_data:
    df = pd.DataFrame(data)
    andhra_df = pd.concat([andhra_df, df], ignore_index=True)

andhra_df.to_csv('andhra.csv')

In [39]:
def ratings(x):
    if x >= 4:
        return '4 to 5'
    elif x <4 and x >=3:
        return '3 to 4'
    else:
        return 'Below 3'
andhra_df.Ratings = pd.to_numeric(andhra_df.Ratings)
andhra_df['Ratings_range'] = andhra_df['Ratings'].apply(ratings)

In [44]:
def fare_segment(x):
    if x >= 3000:
        return "3000 Above"
    if x > 1000 and x <3000:
        return '1000 to 3000'
    elif x > 500 and x<1000:
        return 'below 1000'
    else:
        return 'others'

andhra_df.Bus_fare = andhra_df.Bus_fare.str.replace('INR','',regex= True).astype(float)
#andhra_df.Bus_fare = pd.to_numeric(andhra_df.Bus_fare)
andhra_df['Bus_fare_range'] = andhra_df['Bus_fare'].apply(fare_segment)

In [57]:
andhra_df.Bus_types.dtype

dtype('O')

In [59]:
import re

# Define the function to categorize bus types
def ac_non(bus_type):
    if isinstance(bus_type,str):
        pattern = '^NON'
        if re.findall(pattern, bus_type):
            return 'Non A/C'
        else:
            return 'A/C'

# Apply the function to the 'Bus_types' column
andhra_df['AC_Type'] = andhra_df['Bus_types'].apply(ac_non)

# View the result
andhra_df.AC_Type.value_counts()

AC_Type
A/C        2653
Non A/C     310
Name: count, dtype: int64

In [64]:
import re

# Define the seat categorization function
def seat(bus_type):
    if isinstance(bus_type, str):
        if re.search(r'\bSleeper\b', bus_type):
            return 'Sleeper'
        elif re.search(r'\bSeater\b', bus_type):
            return 'Seater'
        elif re.search(r'\bSemi Sleeper\b', bus_type):
            return 'Semi Sleeper'
        elif re.search(r'\bPush Back\b', bus_type):
            return 'Push Back'
    return 'Other'

# Apply the function to the Bus_types column
andhra_df['Seat_Type'] = andhra_df['Bus_types'].apply(seat)

# View the resul
andhra_df.Seat_Type.value_counts()


Seat_Type
Sleeper      2823
Seater        111
Other         109
Push Back      15
Name: count, dtype: int64

In [53]:
def categorize_bus_type(bus_type):
    if isinstance(bus_type, str):
        if 'A/C' in bus_type and 'Sleeper' in bus_type:
            return 'AC Sleeper'
        elif 'A/C' in bus_type and ('Semi Sleeper' in bus_type or 'Push Back' in bus_type):
            return 'AC Semi Sleeper'
        elif 'A/C' in bus_type and 'Seater' in bus_type:
            return 'AC Seater'
        elif 'NON A/C' in bus_type and 'Sleeper' in bus_type:
            return 'Non AC Sleeper'
        elif 'NON A/C' in bus_type and ('Semi Sleeper' in bus_type or 'Push Back' in bus_type):
            return 'Non AC Semi Sleeper'
        elif 'NON A/C' in bus_type and 'Seater' in bus_type:
            return 'Non AC Seater'
    return 'Other'

# Apply the mapping function to create a new column
andhra_df['Bus_category'] = andhra_df['Bus_types'].apply(categorize_bus_type)

In [68]:
# Function to categorize time into hourly intervals
def categorize_hourly_interval(time_str):
    try:
        time_obj = pd.to_datetime(time_str, format='%H:%M')
        hour = time_obj.hour
        if hour == 23:
            return '23:00 - 00:00'
        else:
            return f'{hour:02d}:00 - {hour+1:02d}:00'
    except Exception as e:
        return 'Invalid Time'

# Apply the function to both Starting_time and Ending_time
andhra_df['Starting_time_interval'] = andhra_df['Starting_time'].apply(categorize_hourly_interval)
andhra_df['Ending_time_interval'] = andhra_df['Ending_time'].apply(categorize_hourly_interval)


In [75]:
andhra_df['Route'] = andhra_df['Starting_names'] + ' ' +'to' + ' ' + andhra_df['Reaching_names']

In [78]:
andhra_df.Route.value_counts()

Route
Hyderabad to Vijayawada             307
Hyderabad to Kurnool                194
Vijayawada to Hyderabad             177
Bangalore to Tirupati               172
Visakhapatnam to Vijayawada         152
Hyderabad to Anantapur (andhr...    143
Eluru to Hyderabad                  132
Vijayawada to Visakhapatnam         132
Bangalore to Anantapur (andhr...    114
Rajahmundry to Hyderabad            110
Vijayawada to Tirupati              101
Hyderabad to Ongole                 100
Ongole to Hyderabad                  86
Nellore to Bangalore                 82
Visakhapatnam to Rajahmundry         81
Tirupati to Bangalore                57
Bangalore to Chittoor (Andhra...     56
Rajahmundry to Vijayawada            55
Nandyal to Hyderabad                 55
Vijayawada to Rajahmundry            51
Hyderabad to Guntur (Andhra P...     51
Anantapur (andhr... to Hyderabad     50
Bangalore to Kadapa                  46
Kadapa to Bangalore                  45
Guntur (Andhra P... to Hyderabad  

In [55]:
andhra_df.Bus_category.value_counts()

Bus_category
AC Sleeper         2703
Other               238
AC Seater            97
AC Semi Sleeper      20
Name: count, dtype: int64

In [67]:
andhra_df.Starting_time.dtype

dtype('O')

In [77]:
andhra_df.to_csv('andhra.csv')

In [32]:
andhra_df.Bus_types.unique()

array(['Electric A/C Seater (2+2)', 'Electric A/C Seater/Sleeper (2+1)',
       'NON A/C Seater Push Back (2+2)', 'A/C Seater / Sleeper (2+1)',
       'Non A/C Seater / Sleeper (2+1)', 'NON A/C Airbus (2+2)',
       'NON A/C Sleeper (2+1)', 'A/C Sleeper (2+1)', 'AC Sleeper (2+1)',
       'Volvo A/C Semi Sleeper (2+2)', 'Bharat Benz A/C Sleeper (2+1)',
       'Volvo Multi-Axle Sleeper A/C (2+1)',
       'NON A/C Seater/ Sleeper (2+1)',
       'Scania AC Multi Axle Sleeper (2+1)', 'A/C Seater (2+2)',
       'Volvo Multi Axle A/C Sleeper I-Shift B11R (2+1)',
       'NON AC Seater / Sleeper 2+1',
       'Bharat Benz A/C Seater /Sleeper (2+1)',
       'Mercedes Benz Multi-Axle A/C Sleeper (2+1)',
       'Volvo 9600 Multi-Axle A/C Sleeper (2+1)',
       'A/C Volvo B11R Multi-Axle Sleeper (2+1)',
       'NON A/C Executive (2+1)', 'VE A/C Sleeper (2+1)',
       'Volvo A/C Sleeper (2+1)', 'Non AC Seater (2+2)',
       'NON A/C Seater (2+2)', 'Scania Multi-Axle AC Semi Sleeper (2+2)',
       'Vo

In [35]:
andhra_df[andhra_df['Bus_types']=='NON A/C Seater (2+3)']

,Starting_names,Reaching_names,Names,Bus_fare,Ratings,Seats,Bus_types,Starting_time,Duration,Ending_time,Starting_point
1351,Rayachoti,Bangalore,SUVARNAMUKHI TRAVELS,INR 280,3.0,5 Seats available,NON A/C Seater (2+3),22:15,06h 00m,04:15,Netaji Circle Banglow
1373,Rayachoti,Bangalore,SUVARNAMUKHI TRAVELS,INR 280,2.7,9 Seats available,NON A/C Seater (2+3),23:00,05h 30m,04:30,Rayachoti Bus Stand
1773,Bangalore,Rayachoti,Sri Vinayaka Travels,INR 350,4.0,29 Seats available,NON A/C Seater (2+3),21:30,06h 00m,03:30,Kalasipalyam
1818,Kadapa,Bangalore,SUVARNAMUKHI TRAVELS,INR 300,3.0,9 Seats available,NON A/C Seater (2+3),21:45,06h 45m,04:30,Suvarnamukhi Travels
1821,Kadapa,Bangalore,SUVARNAMUKHI TRAVELS,INR 300,2.8,5 Seats available,NON A/C Seater (2+3),21:00,07h 15m,04:15,Suvarnamukhi Travels
2483,Bangalore,Madanapalli,Sri Vinayaka Travels,INR 350,3.9,16 Seats available,NON A/C Seater (2+3),21:30,04h 00m,01:30,Chandapura
2706,Bangalore,Kadapa,Sri Vinayaka Travels,INR 350,3.0,29 Seats available,NON A/C Seater (2+3),21:30,07h 30m,05:00,Kalasipalyam


In [99]:
len(all_route_links)

49

In [18]:
# Kerala dataset


# Initialize the WebDriver
driver = webdriver.Chrome()
driver.maximize_window()
driver.implicitly_wait(10)
driver.get("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile")

wait = WebDriverWait(driver, 20)

# Function to collect route links from the current page
def collect_route_links():
    route_details_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
    route_details_links = [element.get_attribute('href') for element in route_details_elements]
    return route_details_links

# Collect route links from the first page
all_route_links = collect_route_links()

# Iterate over pagination pages to collect more route links
page_number = 1
while True:
    try:
        # Locate the pagination container
        pagination_container = wait.until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
        ))

        # Locate the next page button within the container
        next_page_button = pagination_container.find_element(
            By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
        )
        
        # Scroll the next page button into view
        driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
        time.sleep(1)  # Wait for a bit after scrolling
        
        # Log the action
        print(f"Clicking on page {page_number + 1}")
        
        # Click the next page button
        next_page_button.click()
        
        # Wait for the page number to update to the next page
        wait.until(EC.text_to_be_present_in_element(
            (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
        
        # Log the successful page navigation
        print(f"Successfully navigated to page {page_number + 1}")
        
        # Wait for a short duration to ensure the next page loads completely
        time.sleep(3)

        # Collect route links from the current page
        all_route_links.extend(collect_route_links())
        
        page_number += 1

    except Exception as e:
        print(f"No more pages to navigate or an error occurred: {e}")
        break

# Remove duplicates, if any
all_route_links = list(set(all_route_links))

# Display the collected route links
print(f"Collected {len(all_route_links)} route links:")

all_data = []
for link in all_route_links:
        driver.get(link)
        time.sleep(5)  # Adjust this sleep time as needed

        # Scroll to the bottom to load all content
        loading_waiting_time = 1
        previous_page_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(loading_waiting_time)
            new_page_height = driver.execute_script("return document.body.scrollHeight")
            if new_page_height == previous_page_height:
                break
            previous_page_height = new_page_height

        bus_details = collect_data()
        all_data.append(bus_details)

# Close the WebDriver
driver.quit()


Clicking on page 2
Successfully navigated to page 2
No more pages to navigate or an error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7A7FB3E32+31618]
	(No symbol) [0x00007FF7A7F2B099]
	(No symbol) [0x00007FF7A7DE888A]
	(No symbol) [0x00007FF7A7E38524]
	(No symbol) [0x00007FF7A7E3862C]
	(No symbol) [0x00007FF7A7E7F787]
	(No symbol) [0x00007FF7A7E5D14F]
	(No symbol) [0x00007FF7A7E7CA80]
	(No symbol) [0x00007FF7A7E5CEB3]
	(No symbol) [0x00007FF7A7E2A46B]
	(No symbol) [0x00007FF7A7E2B001]
	GetHandleVerifier [0x00007FF7A82B9FFD+3202381]
	GetHandleVerifier [0x00007FF7A8306A1D+3516269]
	GetHandleVerifier [0x00007FF7A82FC490+3473888]
	GetHandleVerifier [0x00007FF7A8065D36+760454]
	(No symbol) [0x00007FF7A7F36B3F]
	(No symbol) [0x00007FF7A7F31CD4]
	(No symbol) [0x00007FF7A7F31E62]
	(No symbol) [0x00007FF7A7F2120F]
	(No symbol) [0x00007FF978A0257D]
	RtlUserThreadStart [0x00007FF97992AF28+40]

Collected 19 route links:


NameError: name 'Starting_names' is not defined

In [11]:
kerala_df = pd.DataFrame()

for data in all_data:
    df = pd.DataFrame(data)
    kerala_df = pd.concat([kerala_df, df], ignore_index=True)

#kerala_df.to_csv('kerala.csv')

In [12]:
kerala_df

,Route_names,Names,Bus_fare,Ratings,Seats,Bus_types,Starting_time,Duration,Ending_time,Starting_point
0,Kottayam,EMERALD TRAVELS,INR 762,2.1,29 Seats available,Bharat Benz A/C Semi Sleeper (2+2),23:00,05h 30m,04:30,Baker Junction
1,NaN,A1 Travels,INR 700,1.9,29 Seats available,A/C Seater / Sleeper (2+1),23:59,07h 01m,07:00,Kottayam
2,NaN,Ubc travels,INR 700,3.9,17 Seats available,A/C Semi Sleeper (2+2),17:30,07h 30m,01:00,Nagampadam Bus Stand
3,NaN,Ubc travels,INR 900,3.9,24 Seats available,A/C Seater/Sleeper (2+1),18:00,09h 00m,03:00,Nagampadam Bus Stand
4,NaN,Kallada Travels (Suresh Kallada),INR 1100,3.2,10 Seats available,A/C Sleeper (2+1),18:15,07h 10m,01:25,Kallada Travels
...,...,...,...,...,...,...,...,...,...,...
432,NaN,Yathra Logistics,INR 1249,3.1,44 Seats available,NON AC Seater / Sleeper 2+1,22:30,06h 00m,04:30,NaN
433,NaN,National travels and logistics,INR 2890,3.0,NaN,Bharat Benz A/C Sleeper (2+1),23:30,06h 30m,06:00,NaN
434,NaN,Loyal Travels,INR 2500,2.5,NaN,Non A/C Seater / Sleeper (2+1),23:45,05h 00m,04:45,NaN
435,NaN,Maaruti Travels,INR 1249,NaN,NaN,NON A/C Semi Sleeper / Sleeper (2+1),22:30,06h 45m,05:15,NaN


In [108]:
# RSRTC dataset


# Initialize the WebDriver
driver = webdriver.Chrome()
driver.maximize_window()
driver.implicitly_wait(10)
driver.get("https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile")

wait = WebDriverWait(driver, 20)

# Function to collect route links from the current page
def collect_route_links():
    route_details_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
    route_details_links = [element.get_attribute('href') for element in route_details_elements]
    return route_details_links

# Collect route links from the first page
all_route_links = collect_route_links()

# Iterate over pagination pages to collect more route links
page_number = 1
while True:
    try:
        # Locate the pagination container
        pagination_container = wait.until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
        ))

        # Locate the next page button within the container
        next_page_button = pagination_container.find_element(
            By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
        )
        
        # Scroll the next page button into view
        driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
        time.sleep(1)  # Wait for a bit after scrolling
        
        # Log the action
        print(f"Clicking on page {page_number + 1}")
        
        # Click the next page button
        next_page_button.click()
        
        # Wait for the page number to update to the next page
        wait.until(EC.text_to_be_present_in_element(
            (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
        
        # Log the successful page navigation
        print(f"Successfully navigated to page {page_number + 1}")
        
        # Wait for a short duration to ensure the next page loads completely
        time.sleep(3)

        # Collect route links from the current page
        all_route_links.extend(collect_route_links())
        
        page_number += 1

    except Exception as e:
        print(f"No more pages to navigate or an error occurred: {e}")
        break

# Remove duplicates, if any
all_route_links = list(set(all_route_links))

# Display the collected route links
print(f"Collected {len(all_route_links)} route links:")

all_data = []
for link in all_route_links:
        driver.get(link)
        time.sleep(5)  # Adjust this sleep time as needed

        # Scroll to the bottom to load all content
        loading_waiting_time = 1
        previous_page_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(loading_waiting_time)
            new_page_height = driver.execute_script("return document.body.scrollHeight")
            if new_page_height == previous_page_height:
                break
            previous_page_height = new_page_height

        bus_details = collect_data()
        all_data.append(bus_details)

# Close the WebDriver
driver.quit()


Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
No more pages to navigate or an error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6EAA73E42+31618]
	(No symbol) [0x00007FF6EA9EB0A9]
	(No symbol) [0x00007FF6EA8A888A]
	(No symbol) [0x00007FF6EA8F8524]
	(No symbol) [0x00007FF6EA8F862C]
	(No symbol) [0x00007FF6EA93F787]
	(No symbol) [0x00007FF6EA91D14F]
	(No symbol) [0x00007FF6EA93CA80]
	(No symbol) [0x00007FF6EA91CEB3]
	(No symbol) [0x00007FF6EA8EA46B]
	(No symbol) [0x00007FF6EA8EB001]
	GetHandleVerifier [0x00007FF6EAD7A01D+3202397]
	GetHandleVerifier [0x00007FF6EADC6A3D+3516285]
	GetHandleVerifier [0x00007FF6EADBC4B0+3473904]
	GetHandleVerifier [0x00007FF6EAB25D46+760454]
	(No symbol) [0x00007FF6EA9F6B4F]
	(No symbol) [0x00007FF6EA9F1CE4]
	(No symbol) [0x00007FF6EA9F1E72]
	(No symbol) [0x00007FF6EA9E121F]
	(No symbol) [0x00007FF978A0257D]
	RtlUserThreadStart [0x00007FF97992AF28+40]

Collected 21 route links:


In [109]:
rsrtc_df = pd.DataFrame()

for data in all_data:
    df = pd.DataFrame(data)
    rsrtc_df = pd.concat([rsrtc_df, df], ignore_index=True)

rsrtc_df.to_csv('rsrtc.csv')

In [111]:
# KTCL dataset


# Initialize the WebDriver
driver = webdriver.Chrome()
driver.maximize_window()
driver.implicitly_wait(10)
driver.get("https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile")

wait = WebDriverWait(driver, 20)

# Function to collect route links from the current page
def collect_route_links():
    route_details_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
    route_details_links = [element.get_attribute('href') for element in route_details_elements]
    return route_details_links

# Collect route links from the first page
all_route_links = collect_route_links()

# Iterate over pagination pages to collect more route links
page_number = 1
while True:
    try:
        # Locate the pagination container
        pagination_container = wait.until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
        ))

        # Locate the next page button within the container
        next_page_button = pagination_container.find_element(
            By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
        )
        
        # Scroll the next page button into view
        driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
        time.sleep(1)  # Wait for a bit after scrolling
        
        # Log the action
        print(f"Clicking on page {page_number + 1}")
        
        # Click the next page button
        next_page_button.click()
        
        # Wait for the page number to update to the next page
        wait.until(EC.text_to_be_present_in_element(
            (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
        
        # Log the successful page navigation
        print(f"Successfully navigated to page {page_number + 1}")
        
        # Wait for a short duration to ensure the next page loads completely
        time.sleep(3)

        # Collect route links from the current page
        all_route_links.extend(collect_route_links())
        
        page_number += 1

    except Exception as e:
        print(f"No more pages to navigate or an error occurred: {e}")
        break

# Remove duplicates, if any
all_route_links = list(set(all_route_links))

# Display the collected route links
print(f"Collected {len(all_route_links)} route links:")

all_data = []
for link in all_route_links:
        driver.get(link)
        time.sleep(5)  # Adjust this sleep time as needed

        # Scroll to the bottom to load all content
        loading_waiting_time = 1
        previous_page_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(loading_waiting_time)
            new_page_height = driver.execute_script("return document.body.scrollHeight")
            if new_page_height == previous_page_height:
                break
            previous_page_height = new_page_height

        bus_details = collect_data()
        all_data.append(bus_details)

# Close the WebDriver
driver.quit()


Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
No more pages to navigate or an error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class, "DC_117_pageTabs") and text()="5"]"}
  (Session info: chrome=126.0.6478.63); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6EAA73E42+31618]
	(No symbol) [0x00007FF6EA9EB0A9]
	(No symbol) [0x00007FF6EA8A888A]
	(No symbol) [0x00007FF6EA8F8524]
	(No symbol) [0x00007FF6EA8F862C]
	(No symbol) [0x00007FF6EA8EBCDC]
	(No symbol) [0x00007FF6EA91D14F]
	(No symbol) [0x00007FF6EA8EBBA6]
	(No symbol) [0x00007FF6EA91D320]
	(No symbol) [0x00007FF6EA93CA80]
	(No symbol) [0x00007FF6EA91CEB3]
	(No symbol) [0x00007FF6EA8EA46B]
	(No symbol) [0x00007FF6EA8EB001]
	G

In [112]:
ktcl_df = pd.DataFrame()

for data in all_data:
    df = pd.DataFrame(data)
    ktcl_df = pd.concat([ktcl_df, df], ignore_index=True)

ktcl_df.to_csv('ktcl.csv')

In [113]:
ktcl_df

,Names,Bus_fare,Ratings,Seats,Bus_types,Starting_time,Duration,Ending_time,Starting_point
0,Kadamba Transport Corporation Limited (KTCL) -...,INR 735,3.3,28 Seats available,Non AC Seater 2+2,19:00,12h 00m,07:00,Vasco
1,Konkan Tours & Travels,INR 3000,3.5,32 Seats available,NON A/C Sleeper (2+1),17:00,11h 00m,04:00,Panjim
2,Zingbus Plus,924,4.4,24 Seats available,Bharat Benz A/C Sleeper (2+1),17:00,15h 00m,08:40,Borivali West
3,Zingbus Plus,827,4.6,19 Seats available,VE A/C Sleeper (2+1),20:00,15h 20m,11:20,Borivali West
4,IntrCity SmartBus,INR 884,4.4,8 Seats available,AC Sleeper (2+1),17:00,15h 15m,08:15,Borivali West
...,...,...,...,...,...,...,...,...,...
557,Konkan Tours & Travels,INR 3000,3.5,32 Seats available,NON A/C Sleeper (2+1),18:50,12h 10m,07:00,Solapur Bypass
558,Kadamba Transport Corporation Limited (KTCL) -...,INR 200,4.8,NaN,AC Seater (2+2),18:30,01h 30m,20:00,NaN
559,Kadamba Transport Corporation Limited (KTCL) -...,INR 200,4.0,NaN,AC Seater (2+2),15:30,01h 45m,17:15,NaN
560,Kadamba Transport Corporation Limited (KTCL) -...,INR 200,3.9,NaN,Volvo AC Seater 2+2,13:30,01h 45m,15:15,NaN


In [5]:
# SBSTC dataset


# Initialize the WebDriver
driver = webdriver.Chrome()
driver.maximize_window()
driver.implicitly_wait(10)
driver.get("https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile")

wait = WebDriverWait(driver, 20)

# Function to collect route links from the current page
def collect_route_links():
    route_details_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
    route_details_links = [element.get_attribute('href') for element in route_details_elements]
    return route_details_links

# Collect route links from the first page
all_route_links = collect_route_links()

# Iterate over pagination pages to collect more route links
page_number = 1
while True:
    try:
        # Locate the pagination container
        pagination_container = wait.until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
        ))

        # Locate the next page button within the container
        next_page_button = pagination_container.find_element(
            By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
        )
        
        # Scroll the next page button into view
        driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
        time.sleep(1)  # Wait for a bit after scrolling
        
        # Log the action
        print(f"Clicking on page {page_number + 1}")
        
        # Click the next page button
        next_page_button.click()
        
        # Wait for the page number to update to the next page
        wait.until(EC.text_to_be_present_in_element(
            (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
        
        # Log the successful page navigation
        print(f"Successfully navigated to page {page_number + 1}")
        
        # Wait for a short duration to ensure the next page loads completely
        time.sleep(3)

        # Collect route links from the current page
        all_route_links.extend(collect_route_links())
        
        page_number += 1

    except Exception as e:
        print(f"No more pages to navigate or an error occurred: {e}")
        break

# Remove duplicates, if any
all_route_links = list(set(all_route_links))

# Display the collected route links
print(f"Collected {len(all_route_links)} route links:")

all_data = []
for link in all_route_links:
        driver.get(link)
        time.sleep(5)  # Adjust this sleep time as needed

        # Scroll to the bottom to load all content
        loading_waiting_time = 1
        previous_page_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(loading_waiting_time)
            new_page_height = driver.execute_script("return document.body.scrollHeight")
            if new_page_height == previous_page_height:
                break
            previous_page_height = new_page_height

        bus_details = collect_data()
        all_data.append(bus_details)

# Close the WebDriver
driver.quit()


Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
Clicking on page 5
Successfully navigated to page 5
No more pages to navigate or an error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF697B23E42+31618]
	(No symbol) [0x00007FF697A9B0A9]
	(No symbol) [0x00007FF69795888A]
	(No symbol) [0x00007FF6979A8524]
	(No symbol) [0x00007FF6979A862C]
	(No symbol) [0x00007FF6979EF787]
	(No symbol) [0x00007FF6979CD14F]
	(No symbol) [0x00007FF6979ECA80]
	(No symbol) [0x00007FF6979CCEB3]
	(No symbol) [0x00007FF69799A46B]
	(No symbol) [0x00007FF69799B001]
	GetHandleVerifier [0x00007FF697E2A01D+3202397]
	GetHandleVerifier [0x00007FF697E76A3D+3516285]
	GetHandleVerifier [0x00007FF697E6C4B0+3473904]
	GetHandleVerifier [0x00007FF697BD5D46+760454]
	(No symbol) [0x00007FF697AA6B4F]
	(No symbol) [0x00007FF697AA1CE4]
	(No symbol) [0x00007FF697AA1E72]
	(No symbol) [0x00007FF697A9121F]
	(No symb

In [6]:
SBSTC_df = pd.DataFrame()

for data in all_data:
    df = pd.DataFrame(data)
    SBSTC_df = pd.concat([SBSTC_df, df], ignore_index=True)

SBSTC_df.to_csv('SBSTC.csv')

In [8]:
# SBSTC dataset


# Initialize the WebDriver
driver = webdriver.Chrome()
driver.maximize_window()
driver.implicitly_wait(10)
driver.get("https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile")

wait = WebDriverWait(driver, 20)

# Function to collect route links from the current page
def collect_route_links():
    route_details_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
    route_details_links = [element.get_attribute('href') for element in route_details_elements]
    return route_details_links

# Collect route links from the first page
all_route_links = collect_route_links()

# Iterate over pagination pages to collect more route links
page_number = 1
while True:
    try:
        # Locate the pagination container
        pagination_container = wait.until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
        ))

        # Locate the next page button within the container
        next_page_button = pagination_container.find_element(
            By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
        )
        
        # Scroll the next page button into view
        driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
        time.sleep(1)  # Wait for a bit after scrolling
        
        # Log the action
        print(f"Clicking on page {page_number + 1}")
        
        # Click the next page button
        next_page_button.click()
        
        # Wait for the page number to update to the next page
        wait.until(EC.text_to_be_present_in_element(
            (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
        
        # Log the successful page navigation
        print(f"Successfully navigated to page {page_number + 1}")
        
        # Wait for a short duration to ensure the next page loads completely
        time.sleep(3)

        # Collect route links from the current page
        all_route_links.extend(collect_route_links())
        
        page_number += 1

    except Exception as e:
        print(f"No more pages to navigate or an error occurred: {e}")
        break

# Remove duplicates, if any
all_route_links = list(set(all_route_links))

# Display the collected route links
print(f"Collected {len(all_route_links)} route links:")

all_data = []
for link in all_route_links:
        driver.get(link)
        time.sleep(5)  # Adjust this sleep time as needed

        # Scroll to the bottom to load all content
        loading_waiting_time = 1
        previous_page_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(loading_waiting_time)
            new_page_height = driver.execute_script("return document.body.scrollHeight")
            if new_page_height == previous_page_height:
                break
            previous_page_height = new_page_height

        bus_details = collect_data()
        all_data.append(bus_details)

# Close the WebDriver
driver.quit()


Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
Clicking on page 5
Successfully navigated to page 5
No more pages to navigate or an error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF697B23E42+31618]
	(No symbol) [0x00007FF697A9B0A9]
	(No symbol) [0x00007FF69795888A]
	(No symbol) [0x00007FF6979A8524]
	(No symbol) [0x00007FF6979A862C]
	(No symbol) [0x00007FF6979EF787]
	(No symbol) [0x00007FF6979CD14F]
	(No symbol) [0x00007FF6979ECA80]
	(No symbol) [0x00007FF6979CCEB3]
	(No symbol) [0x00007FF69799A46B]
	(No symbol) [0x00007FF69799B001]
	GetHandleVerifier [0x00007FF697E2A01D+3202397]
	GetHandleVerifier [0x00007FF697E76A3D+3516285]
	GetHandleVerifier [0x00007FF697E6C4B0+3473904]
	GetHandleVerifier [0x00007FF697BD5D46+760454]
	(No symbol) [0x00007FF697AA6B4F]
	(No symbol) [0x00007FF697AA1CE4]
	(No symbol) [0x00007FF697AA1E72]
	(No symbol) [0x00007FF697A9121F]
	(No symb

In [9]:
HRTC_df = pd.DataFrame()

for data in all_data:
    df = pd.DataFrame(data)
    HRTC_df = pd.concat([HRTC_df, df], ignore_index=True)

HRTC_df.to_csv('HRTC.csv')

In [11]:
# ASTC dataset


# Initialize the WebDriver
driver = webdriver.Chrome()
driver.maximize_window()
driver.implicitly_wait(10)
driver.get("https://www.redbus.in/online-booking/astc/?utm_source=rtchometile")

wait = WebDriverWait(driver, 20)

# Function to collect route links from the current page
def collect_route_links():
    route_details_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
    route_details_links = [element.get_attribute('href') for element in route_details_elements]
    return route_details_links

# Collect route links from the first page
all_route_links = collect_route_links()

# Iterate over pagination pages to collect more route links
page_number = 1
while True:
    try:
        # Locate the pagination container
        pagination_container = wait.until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
        ))

        # Locate the next page button within the container
        next_page_button = pagination_container.find_element(
            By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
        )
        
        # Scroll the next page button into view
        driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
        time.sleep(1)  # Wait for a bit after scrolling
        
        # Log the action
        print(f"Clicking on page {page_number + 1}")
        
        # Click the next page button
        next_page_button.click()
        
        # Wait for the page number to update to the next page
        wait.until(EC.text_to_be_present_in_element(
            (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
        
        # Log the successful page navigation
        print(f"Successfully navigated to page {page_number + 1}")
        
        # Wait for a short duration to ensure the next page loads completely
        time.sleep(3)

        # Collect route links from the current page
        all_route_links.extend(collect_route_links())
        
        page_number += 1

    except Exception as e:
        print(f"No more pages to navigate or an error occurred: {e}")
        break

# Remove duplicates, if any
all_route_links = list(set(all_route_links))

# Display the collected route links
print(f"Collected {len(all_route_links)} route links:")

all_data = []
for link in all_route_links:
        driver.get(link)
        time.sleep(5)  # Adjust this sleep time as needed

        # Scroll to the bottom to load all content
        loading_waiting_time = 1
        previous_page_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(loading_waiting_time)
            new_page_height = driver.execute_script("return document.body.scrollHeight")
            if new_page_height == previous_page_height:
                break
            previous_page_height = new_page_height

        bus_details = collect_data()
        all_data.append(bus_details)

# Close the WebDriver
driver.quit()


Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
Clicking on page 5
Successfully navigated to page 5
No more pages to navigate or an error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF697B23E42+31618]
	(No symbol) [0x00007FF697A9B0A9]
	(No symbol) [0x00007FF69795888A]
	(No symbol) [0x00007FF6979A8524]
	(No symbol) [0x00007FF6979A862C]
	(No symbol) [0x00007FF6979EF787]
	(No symbol) [0x00007FF6979CD14F]
	(No symbol) [0x00007FF6979ECA80]
	(No symbol) [0x00007FF6979CCEB3]
	(No symbol) [0x00007FF69799A46B]
	(No symbol) [0x00007FF69799B001]
	GetHandleVerifier [0x00007FF697E2A01D+3202397]
	GetHandleVerifier [0x00007FF697E76A3D+3516285]
	GetHandleVerifier [0x00007FF697E6C4B0+3473904]
	GetHandleVerifier [0x00007FF697BD5D46+760454]
	(No symbol) [0x00007FF697AA6B4F]
	(No symbol) [0x00007FF697AA1CE4]
	(No symbol) [0x00007FF697AA1E72]
	(No symbol) [0x00007FF697A9121F]
	(No symb

In [12]:
ASTC_df = pd.DataFrame()

for data in all_data:
    df = pd.DataFrame(data)
    ASTC_df = pd.concat([ASTC_df, df], ignore_index=True)

ASTC_df.to_csv('ASTC.csv')

In [14]:
# UPSRTC dataset


# Initialize the WebDriver
driver = webdriver.Chrome()
driver.maximize_window()
driver.implicitly_wait(10)
driver.get("https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile")

wait = WebDriverWait(driver, 20)

# Function to collect route links from the current page
def collect_route_links():
    route_details_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
    route_details_links = [element.get_attribute('href') for element in route_details_elements]
    return route_details_links

# Collect route links from the first page
all_route_links = collect_route_links()

# Iterate over pagination pages to collect more route links
page_number = 1
while True:
    try:
        # Locate the pagination container
        pagination_container = wait.until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
        ))

        # Locate the next page button within the container
        next_page_button = pagination_container.find_element(
            By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
        )
        
        # Scroll the next page button into view
        driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
        time.sleep(1)  # Wait for a bit after scrolling
        
        # Log the action
        print(f"Clicking on page {page_number + 1}")
        
        # Click the next page button
        next_page_button.click()
        
        # Wait for the page number to update to the next page
        wait.until(EC.text_to_be_present_in_element(
            (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
        
        # Log the successful page navigation
        print(f"Successfully navigated to page {page_number + 1}")
        
        # Wait for a short duration to ensure the next page loads completely
        time.sleep(3)

        # Collect route links from the current page
        all_route_links.extend(collect_route_links())
        
        page_number += 1

    except Exception as e:
        print(f"No more pages to navigate or an error occurred: {e}")
        break

# Remove duplicates, if any
all_route_links = list(set(all_route_links))

# Display the collected route links
print(f"Collected {len(all_route_links)} route links:")

all_data = []
for link in all_route_links:
        driver.get(link)
        time.sleep(5)  # Adjust this sleep time as needed

        # Scroll to the bottom to load all content
        loading_waiting_time = 1
        previous_page_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(loading_waiting_time)
            new_page_height = driver.execute_script("return document.body.scrollHeight")
            if new_page_height == previous_page_height:
                break
            previous_page_height = new_page_height

        bus_details = collect_data()
        all_data.append(bus_details)

# Close the WebDriver
driver.quit()


Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
Clicking on page 5
Successfully navigated to page 5
No more pages to navigate or an error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF697B23E42+31618]
	(No symbol) [0x00007FF697A9B0A9]
	(No symbol) [0x00007FF69795888A]
	(No symbol) [0x00007FF6979A8524]
	(No symbol) [0x00007FF6979A862C]
	(No symbol) [0x00007FF6979EF787]
	(No symbol) [0x00007FF6979CD14F]
	(No symbol) [0x00007FF6979ECA80]
	(No symbol) [0x00007FF6979CCEB3]
	(No symbol) [0x00007FF69799A46B]
	(No symbol) [0x00007FF69799B001]
	GetHandleVerifier [0x00007FF697E2A01D+3202397]
	GetHandleVerifier [0x00007FF697E76A3D+3516285]
	GetHandleVerifier [0x00007FF697E6C4B0+3473904]
	GetHandleVerifier [0x00007FF697BD5D46+760454]
	(No symbol) [0x00007FF697AA6B4F]
	(No symbol) [0x00007FF697AA1CE4]
	(No symbol) [0x00007FF697AA1E72]
	(No symbol) [0x00007FF697A9121F]
	(No symb

In [15]:
UPSRTC_df = pd.DataFrame()

for data in all_data:
    df = pd.DataFrame(data)
    UPSRTC_df = pd.concat([UPSRTC_df, df], ignore_index=True)

UPSRTC_df.to_csv('UPSRTC.csv')

In [16]:
UPSRTC_df

,Names,Bus_fare,Ratings,Seats,Bus_types,Starting_time,Duration,Ending_time,Starting_point
0,Vijay Tour and Travels,INR 799,3.9,6 Seats available,A/C Seater / Sleeper (2+1),19:00,06h 10m,01:10,Vijay travels near satellite ...
1,New Khaira TPT,INR 900,3.7,15 Seats available,AC Sleeper (2+1),23:15,05h 45m,05:00,Outside Satellite Bus Stand
2,New Khaira Transport,INR 500,2.7,7 Seats available,A/C Seater / Sleeper (2+2),23:30,06h 30m,06:00,Near Satelite Bus Stund
3,Vijay Tour and Travels,INR 649,2.9,9 Seats available,A/C Seater / Sleeper (2+1),17:00,06h 20m,23:20,Bareily
4,Rahul Travels,INR 1062,3.6,20 Seats available,AshokLeyland Stile A/C,22:30,06h 30m,05:00,Near Satelite Bus Stund
...,...,...,...,...,...,...,...,...,...
1186,Mahalaxmi Travels,INR 799,NaN,NaN,Bharat Benz A/C Seater /Sleeper (2+1),19:30,03h 15m,22:45,NaN
1187,Shatabdi Travels,INR 888,NaN,NaN,AC Sleeper (2+1),21:01,02h 30m,23:31,NaN
1188,UPSRTC - RKD0039,INR 814,4.0,28 Seats available,Janrath AC Seater 2+2,21:00,08h 00m,05:00,BAREILLY OLD BUS STATION
1189,UPSRTC - VKS0001,INR 830,2.6,16 Seats available,Janrath AC Seater 2+2,22:05,07h 10m,05:15,BAREILLY OLD BUS STATION


In [17]:
# WBTC dataset


# Initialize the WebDriver
driver = webdriver.Chrome()
driver.maximize_window()
driver.implicitly_wait(10)
driver.get("https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile")

wait = WebDriverWait(driver, 20)

# Function to collect route links from the current page
def collect_route_links():
    route_details_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
    route_details_links = [element.get_attribute('href') for element in route_details_elements]
    return route_details_links

# Collect route links from the first page
all_route_links = collect_route_links()

# Iterate over pagination pages to collect more route links
page_number = 1
while True:
    try:
        # Locate the pagination container
        pagination_container = wait.until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
        ))

        # Locate the next page button within the container
        next_page_button = pagination_container.find_element(
            By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
        )
        
        # Scroll the next page button into view
        driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
        time.sleep(1)  # Wait for a bit after scrolling
        
        # Log the action
        print(f"Clicking on page {page_number + 1}")
        
        # Click the next page button
        next_page_button.click()
        
        # Wait for the page number to update to the next page
        wait.until(EC.text_to_be_present_in_element(
            (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
        
        # Log the successful page navigation
        print(f"Successfully navigated to page {page_number + 1}")
        
        # Wait for a short duration to ensure the next page loads completely
        time.sleep(3)

        # Collect route links from the current page
        all_route_links.extend(collect_route_links())
        
        page_number += 1

    except Exception as e:
        print(f"No more pages to navigate or an error occurred: {e}")
        break

# Remove duplicates, if any
all_route_links = list(set(all_route_links))

# Display the collected route links
print(f"Collected {len(all_route_links)} route links:")

all_data = []
for link in all_route_links:
        driver.get(link)
        time.sleep(5)  # Adjust this sleep time as needed

        # Scroll to the bottom to load all content
        loading_waiting_time = 1
        previous_page_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(loading_waiting_time)
            new_page_height = driver.execute_script("return document.body.scrollHeight")
            if new_page_height == previous_page_height:
                break
            previous_page_height = new_page_height

        bus_details = collect_data()
        all_data.append(bus_details)

# Close the WebDriver
driver.quit()


Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
Clicking on page 5
Successfully navigated to page 5
No more pages to navigate or an error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF697B23E42+31618]
	(No symbol) [0x00007FF697A9B0A9]
	(No symbol) [0x00007FF69795888A]
	(No symbol) [0x00007FF6979A8524]
	(No symbol) [0x00007FF6979A862C]
	(No symbol) [0x00007FF6979EF787]
	(No symbol) [0x00007FF6979CD14F]
	(No symbol) [0x00007FF6979ECA80]
	(No symbol) [0x00007FF6979CCEB3]
	(No symbol) [0x00007FF69799A46B]
	(No symbol) [0x00007FF69799B001]
	GetHandleVerifier [0x00007FF697E2A01D+3202397]
	GetHandleVerifier [0x00007FF697E76A3D+3516285]
	GetHandleVerifier [0x00007FF697E6C4B0+3473904]
	GetHandleVerifier [0x00007FF697BD5D46+760454]
	(No symbol) [0x00007FF697AA6B4F]
	(No symbol) [0x00007FF697AA1CE4]
	(No symbol) [0x00007FF697AA1E72]
	(No symbol) [0x00007FF697A9121F]
	(No symb

In [18]:
WBTC_df = pd.DataFrame()

for data in all_data:
    df = pd.DataFrame(data)
    WBTC_df = pd.concat([WBTC_df, df], ignore_index=True)

WBTC_df.to_csv('WBTC.csv')

In [19]:
WBTC_df

,Names,Bus_fare,Ratings,Seats,Bus_types,Starting_time,Duration,Ending_time,Starting_point
0,WBTC (CTC) KOLKATA -BOLPUR - SURI (AC-VOLVO) -...,INR 435,3.9,NaN,Volvo AC Seater (2+2),14:30,04h 10m,18:40,NaN
1,WBTC (CTC) KOLKATA -BOLPUR - SURI (AC-VOLVO) -...,INR 375,3.9,NaN,Volvo AC Seater (2+2),14:30,03h 15m,17:45,NaN
2,SBSTC-BARASAT - DIGHA - VIA - NIBEDITA SETU - ...,INR 93,3.7,44 Seats available,Non AC Seater (2+3),11:15,12h 45m,00:00,BARASAT
3,Snemita Paribahan,INR 370,3.2,NaN,AC Seater (2+3),11:35,06h 20m,17:55,NaN
4,City Express,INR 370,2.6,NaN,AC Seater (2+3),11:35,06h 10m,17:45,NaN
...,...,...,...,...,...,...,...,...,...
473,Express Line (Karunamoyee),INR 524,4.8,NaN,Scania Multi-Axle AC Semi Sleeper (2+2),13:10,05h 00m,18:10,NaN
474,Shyamoli Paribahan Pvt Ltd,INR 524,3.7,NaN,Volvo B11R Multi Axle Seater (2+2),10:00,05h 00m,15:00,NaN
475,Sana Travels,INR 400,1.0,NaN,NON AC Seater / Sleeper 2+1,23:58,05h 02m,05:00,NaN
476,Express Line,INR 524,4.5,NaN,Scania Multi-Axle AC Semi Sleeper (2+2),13:20,05h 10m,18:30,NaN


In [20]:
# CTURTC dataset


# Initialize the WebDriver
driver = webdriver.Chrome()
driver.maximize_window()
driver.implicitly_wait(10)
driver.get("https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu")

wait = WebDriverWait(driver, 20)

# Function to collect route links from the current page
def collect_route_links():
    route_details_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
    route_details_links = [element.get_attribute('href') for element in route_details_elements]
    return route_details_links

# Collect route links from the first page
all_route_links = collect_route_links()

# Iterate over pagination pages to collect more route links
page_number = 1
while True:
    try:
        # Locate the pagination container
        pagination_container = wait.until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
        ))

        # Locate the next page button within the container
        next_page_button = pagination_container.find_element(
            By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
        )
        
        # Scroll the next page button into view
        driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
        time.sleep(1)  # Wait for a bit after scrolling
        
        # Log the action
        print(f"Clicking on page {page_number + 1}")
        
        # Click the next page button
        next_page_button.click()
        
        # Wait for the page number to update to the next page
        wait.until(EC.text_to_be_present_in_element(
            (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
        
        # Log the successful page navigation
        print(f"Successfully navigated to page {page_number + 1}")
        
        # Wait for a short duration to ensure the next page loads completely
        time.sleep(3)

        # Collect route links from the current page
        all_route_links.extend(collect_route_links())
        
        page_number += 1

    except Exception as e:
        print(f"No more pages to navigate or an error occurred: {e}")
        break

# Remove duplicates, if any
all_route_links = list(set(all_route_links))

# Display the collected route links
print(f"Collected {len(all_route_links)} route links:")

all_data = []
for link in all_route_links:
        driver.get(link)
        time.sleep(5)  # Adjust this sleep time as needed

        # Scroll to the bottom to load all content
        loading_waiting_time = 1
        previous_page_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(loading_waiting_time)
            new_page_height = driver.execute_script("return document.body.scrollHeight")
            if new_page_height == previous_page_height:
                break
            previous_page_height = new_page_height

        bus_details = collect_data()
        all_data.append(bus_details)

# Close the WebDriver
driver.quit()


Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
Clicking on page 4
Successfully navigated to page 4
Clicking on page 5
Successfully navigated to page 5
No more pages to navigate or an error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF697B23E42+31618]
	(No symbol) [0x00007FF697A9B0A9]
	(No symbol) [0x00007FF69795888A]
	(No symbol) [0x00007FF6979A8524]
	(No symbol) [0x00007FF6979A862C]
	(No symbol) [0x00007FF6979EF787]
	(No symbol) [0x00007FF6979CD14F]
	(No symbol) [0x00007FF6979ECA80]
	(No symbol) [0x00007FF6979CCEB3]
	(No symbol) [0x00007FF69799A46B]
	(No symbol) [0x00007FF69799B001]
	GetHandleVerifier [0x00007FF697E2A01D+3202397]
	GetHandleVerifier [0x00007FF697E76A3D+3516285]
	GetHandleVerifier [0x00007FF697E6C4B0+3473904]
	GetHandleVerifier [0x00007FF697BD5D46+760454]
	(No symbol) [0x00007FF697AA6B4F]
	(No symbol) [0x00007FF697AA1CE4]
	(No symbol) [0x00007FF697AA1E72]
	(No symbol) [0x00007FF697A9121F]
	(No symb

In [21]:
CTURTC_df = pd.DataFrame()

for data in all_data:
    df = pd.DataFrame(data)
    CTURTC_df = pd.concat([CTURTC_df, df], ignore_index=True)

CTURTC_df.to_csv('WBTC.csv')

In [26]:
!pip install pymongo

In [48]:
# PEPSU dataset


# Initialize the WebDriver
driver = webdriver.Chrome()
driver.maximize_window()
driver.implicitly_wait(10)
driver.get("https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile")

wait = WebDriverWait(driver, 20)

# Function to collect route links from the current page
def collect_route_links():
    route_details_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
    route_details_links = [element.get_attribute('href') for element in route_details_elements]
    return route_details_links

# Collect route links from the first page
all_route_links = collect_route_links()

# Iterate over pagination pages to collect more route links
page_number = 1
while True:
    try:
        # Locate the pagination container
        pagination_container = wait.until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')
        ))

        # Locate the next page button within the container
        next_page_button = pagination_container.find_element(
            By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]'
        )
        
        # Scroll the next page button into view
        driver.execute_script("arguments[0].scrollIntoView();", next_page_button)
        time.sleep(1)  # Wait for a bit after scrolling
        
        # Log the action
        print(f"Clicking on page {page_number + 1}")
        
        # Click the next page button
        next_page_button.click()
        
        # Wait for the page number to update to the next page
        wait.until(EC.text_to_be_present_in_element(
            (By.XPATH, '//div[contains(@class, "DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
        
        # Log the successful page navigation
        print(f"Successfully navigated to page {page_number + 1}")
        
        # Wait for a short duration to ensure the next page loads completely
        time.sleep(3)

        # Collect route links from the current page
        all_route_links.extend(collect_route_links())
        
        page_number += 1

    except Exception as e:
        print(f"No more pages to navigate or an error occurred: {e}")
        break

# Remove duplicates, if any
all_route_links = list(set(all_route_links))

# Display the collected route links
print(f"Collected {len(all_route_links)} route links:")

all_data = []
for link in all_route_links:
        driver.get(link)
        time.sleep(5)  # Adjust this sleep time as needed

        # Scroll to the bottom to load all content
        loading_waiting_time = 1
        previous_page_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(loading_waiting_time)
            new_page_height = driver.execute_script("return document.body.scrollHeight")
            if new_page_height == previous_page_height:
                break
            previous_page_height = new_page_height

        bus_details = collect_data()
        all_data.append(bus_details)

# Close the WebDriver
driver.quit()


Clicking on page 2
Successfully navigated to page 2
Clicking on page 3
Successfully navigated to page 3
No more pages to navigate or an error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF796B33E42+31618]
	(No symbol) [0x00007FF796AAB0A9]
	(No symbol) [0x00007FF79696888A]
	(No symbol) [0x00007FF7969B8524]
	(No symbol) [0x00007FF7969B862C]
	(No symbol) [0x00007FF7969FF787]
	(No symbol) [0x00007FF7969DD14F]
	(No symbol) [0x00007FF7969FCA80]
	(No symbol) [0x00007FF7969DCEB3]
	(No symbol) [0x00007FF7969AA46B]
	(No symbol) [0x00007FF7969AB001]
	GetHandleVerifier [0x00007FF796E3A01D+3202397]
	GetHandleVerifier [0x00007FF796E86A3D+3516285]
	GetHandleVerifier [0x00007FF796E7C4B0+3473904]
	GetHandleVerifier [0x00007FF796BE5D46+760454]
	(No symbol) [0x00007FF796AB6B4F]
	(No symbol) [0x00007FF796AB1CE4]
	(No symbol) [0x00007FF796AB1E72]
	(No symbol) [0x00007FF796AA121F]
	(No symbol) [0x00007FF978A0257D]
	RtlUserThreadStart [0x00007FF97992AF28+40]

Collected 22 route links:


In [49]:
PEPSU_df = pd.DataFrame()

for data in all_data:
    df = pd.DataFrame(data)
    PEPSU_df = pd.concat([PEPSU_df, df], ignore_index=True)

PEPSU_df.to_csv('PEPSU.csv')

In [28]:
!pip install pandas sqlalchemy mysql-connector-python


  Obtaining dependency information for mysql-connector-python from https://files.pythonhosted.org/packages/38/b4/44efe46d5aff497b6009e4755a5e26a99fd8fe2769857dffa21fad941767/mysql_connector_python-8.4.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB 217.9 kB/s eta 0:01:07
   ---------------------------------------- 0.0/14.5 MB 245.8 kB/s eta 0:00:59
    --------------------------------------- 0.2/14.5 MB 908.0 kB/s eta 0:00:16
   - -------------------------------------- 0.5/14.5 MB 1.8 MB/s eta 0:00:08
   - -------------------------------------- 0.5/14.5 MB 1.8 MB/s eta 0:00:08
   -- ------------------------------------- 0.8/14.5 MB 2.3 MB/s eta 0:00:06
   --- ------------------------------------ 1.1/14.5 MB 3.0 MB/s eta 0:00:05
   --- 

In [31]:
!pip install pandas sqlalchemy pymysql


  Obtaining dependency information for pymysql from https://files.pythonhosted.org/packages/0c/94/e4181a1f6286f545507528c78016e00065ea913276888db2262507693ce5/PyMySQL-1.1.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   ------------------ --------------------- 20.5/45.0 kB 330.3 kB/s eta 0:00:01
   --------------------------- ------------ 30.7/45.0 kB 330.3 kB/s eta 0:00:01
   ---------------------------------------- 45.0/45.0 kB 318.6 kB/s eta 0:00:00


In [32]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd

# Define your MySQL credentials
mysql_user = 'root'  # Replace with your MySQL username
mysql_password = 'Ta9ta6me@27'  # Replace with your MySQL password
mysql_host = 'localhost'
mysql_port = 3306
database_name = 'redbus_database'

# Connect to MySQL server
connection = pymysql.connect(
    host=mysql_host,
    user=mysql_user,
    password=mysql_password,
    port=mysql_port
)

cursor = connection.cursor()

# Create database
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database_name}")
cursor.execute(f"USE {database_name}")

# Create tables
create_table_query = """
CREATE TABLE IF NOT EXISTS my_table (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    bus_fare DECIMAL(10, 2),
    rating DECIMAL(2, 1),
    seats INT,
    bus_type VARCHAR(255),
    starting_time TIME,
    duration TIME,
    ending_time TIME,
    starting_point VARCHAR(255)
)
"""
cursor.execute(create_table_query)

# Close the cursor and connection
cursor.close()
connection.close()


In [39]:
!pip install mysql.connector

In [38]:
import mysql.connector as sql

In [40]:
con= mysql.connector.connect(
    host="localhost",
    user="root",
    password="Ta9ta6me@27",
    autocommit=True
    )
print(con)

In [41]:
mycursor=con.cursor()

In [42]:
mycursor.execute("CREATE DATABASE REDBUS")

In [43]:
mycursor.execute("SHOW DATABASES")
for x in mycursor:
    print(x)

('information_schema',)
('my_database',)
('mysql',)
('performance_schema',)
('redbus',)
('redbus_database',)
('sys',)


In [44]:
mycursor.execute("USE REDBUS")

In [45]:
def import_csv_to_mysql(csv_file_path, table_name):
    """
    Imports data from a CSV file into a MySQL table.

    Args:
        csv_file_path (str): Path to the CSV file.
        table_name (str): Name of the MySQL table to import data into.
    """

    try:
        # Read the CSV data
        data = pd.read_csv(csv_file_path)

        # Replace with actual column names in your CSV (adjust data types if needed)
        columns = ", ".join(data.columns.tolist())
        values = ", ".join(["%s"] * len(data.columns))

        # Create the SQL query
        sql = f"""INSERT INTO {table_name} ({columns}) VALUES ({values})"""

        # Iterate through each row in the DataFrame
        for index, row in data.iterrows():
            # Convert row to a list for the SQL query
            row_list = row.tolist()
            cursor.execute(sql, row_list)

        # Commit the changes to the database
        connection.commit()
        print(f"Data from {csv_file_path} imported successfully to {table_name} table.")

    except Exception as e:
        print(f"Error importing data: {e}")


In [47]:
import glob
# Find all CSV files in the current directory (adjust the path if needed)
csv_files = [r"C:\Users\Hp\Selenium\andhra.csv", r"C:\Users\Hp\Selenium\ASTC.csv"]  # Add your CSV file paths here

# Define the table name in your MySQL database (replace with your actual table name)
table_name = "redbus_data"

for csv_file in csv_files:
    import_csv_to_mysql(csv_file, table_name)


Error importing data: Cursor closed
Error importing data: Cursor closed



Examining @/win-64::__archspec==1=x86_64:  75%|#######5  | 3/4 [00:01<00:00,  2.54it/s]
                                                                                       

Determining conflicts:   0%|          | 0/4 [00:00<?, ?it/s]
Examining conflict for mysql-connector-python python:   0%|          | 0/4 [00:00<?, ?it/s]
                                                                                           

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - mysql-connector-python -> python[version='>=3.5,<3.6.0a0|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0']

Your python: python=3.11

If python is on the left-most side of the chain, that's the version you've asked for.
When python appears to the right, that indicates that the thing on the left is somehow
not available for the python version you are constrained to. Note that conda will not
change your python version to a different

Retrieving notices: ...working... done
Solving environment: ...working... unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: ...working... unsuccessful attempt using repodata from current_repodata.json, retrying with next repodata source.
Solving environment: ...working... unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: ...working... 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
failed


In [101]:
df = pd.read_csv('Andhra.csv')
df.head()

,Unnamed: 0,Names,Bus_fare,Ratings,Seats,Bus_types,Starting_time,Duration,Ending_time,Starting_point
0,0,NueGo,314,4.0,33 Seats available,Electric A/C Seater (2+2),05:00,06h 10m,11:10,TIRUPATHI (12:30Am)
1,1,FRESHBUS,489,4.6,39 Seats available,Electric A/C Seater (2+2),05:30,05h 35m,11:05,RTC Bus Stand (Sindhu Travels)
2,2,NueGo,314,3.7,44 Seats available,Electric A/C Seater (2+2),07:00,06h 10m,13:10,RTC Bus Stand (Sindhu Travels)
3,3,AdRajesh Transports,INR 1080,3.9,36 Seats available,A/C Sleeper (2+1),23:36,06h 24m,06:00,RTC Bus Stand (Sindhu Travels)
4,4,Yolo Bus,INR 299,4.9,37 Seats available,Bharat Benz A/C Seater /Sleeper (2+1),07:15,05h 58m,13:13,RTC Bus Stand(NEW DOT TRAVELS)


In [102]:
df.Bus_fare = df.Bus_fare.str.replace('INR','',regex = True).astype(float)
df.Seats = df.Seats.str.replace('Seats available','', regex= True).replace('Seat available','',regex=True).astype(float)

In [103]:
df.tail()

,Unnamed: 0,Names,Bus_fare,Ratings,Seats,Bus_types,Starting_time,Duration,Ending_time,Starting_point
3926,3926,VRCR Travels,525.0,3.8,25.0,A/C Seater / Sleeper (2+1),21:35,03h 10m,00:45,Anna Nagar
3927,3927,CGR Travels,900.0,3.7,10.0,Non A/C Seater / Sleeper (2+1),20:00,05h 30m,01:30,Koyambedu
3928,3928,VRCR Travels,425.0,2.1,17.0,Non A/C Seater / Sleeper (2+1),21:50,03h 30m,01:20,Koyambedu
3929,3929,Meenakshi Travels (RR Group),500.0,2.1,NaN,A/C Sleeper (2+1),19:00,27h 30m,22:30,NaN
3930,3930,Meenakshi Travels (RR Group),500.0,NaN,NaN,NON A/C Sleeper (2+1),19:00,27h 30m,22:30,NaN


In [104]:
df.dtypes

Unnamed: 0          int64
Names              object
Bus_fare          float64
Ratings           float64
Seats             float64
Bus_types          object
Starting_time      object
Duration           object
Ending_time        object
Starting_point     object
dtype: object

In [105]:
df.drop('Unnamed: 0',axis=1)

,Names,Bus_fare,Ratings,Seats,Bus_types,Starting_time,Duration,Ending_time,Starting_point
0,NueGo,314.0,4.0,33.0,Electric A/C Seater (2+2),05:00,06h 10m,11:10,TIRUPATHI (12:30Am)
1,FRESHBUS,489.0,4.6,39.0,Electric A/C Seater (2+2),05:30,05h 35m,11:05,RTC Bus Stand (Sindhu Travels)
2,NueGo,314.0,3.7,44.0,Electric A/C Seater (2+2),07:00,06h 10m,13:10,RTC Bus Stand (Sindhu Travels)
3,AdRajesh Transports,1080.0,3.9,36.0,A/C Sleeper (2+1),23:36,06h 24m,06:00,RTC Bus Stand (Sindhu Travels)
4,Yolo Bus,299.0,4.9,37.0,Bharat Benz A/C Seater /Sleeper (2+1),07:15,05h 58m,13:13,RTC Bus Stand(NEW DOT TRAVELS)
...,...,...,...,...,...,...,...,...,...
3926,VRCR Travels,525.0,3.8,25.0,A/C Seater / Sleeper (2+1),21:35,03h 10m,00:45,Anna Nagar
3927,CGR Travels,900.0,3.7,10.0,Non A/C Seater / Sleeper (2+1),20:00,05h 30m,01:30,Koyambedu
3928,VRCR Travels,425.0,2.1,17.0,Non A/C Seater / Sleeper (2+1),21:50,03h 30m,01:20,Koyambedu
3929,Meenakshi Travels (RR Group),500.0,2.1,NaN,A/C Sleeper (2+1),19:00,27h 30m,22:30,NaN


In [106]:
df['Ending_time'] = pd.to_datetime(df['Ending_time'])
df['Starting_time'] = pd.to_datetime(df['Starting_time'])

C:\Users\Hp\AppData\Local\Temp\ipykernel_9688\457672028.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Ending_time'] = pd.to_datetime(df['Ending_time'])
C:\Users\Hp\AppData\Local\Temp\ipykernel_9688\457672028.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Starting_time'] = pd.to_datetime(df['Starting_time'])


In [107]:
df['Travel_time'] = df['Ending_time'] - df['Starting_time']
df.head()

,Unnamed: 0,Names,Bus_fare,Ratings,Seats,Bus_types,Starting_time,Duration,Ending_time,Starting_point,Travel_time
0,0,NueGo,314.0,4.0,33.0,Electric A/C Seater (2+2),2024-06-20 05:00:00,06h 10m,2024-06-20 11:10:00,TIRUPATHI (12:30Am),0 days 06:10:00
1,1,FRESHBUS,489.0,4.6,39.0,Electric A/C Seater (2+2),2024-06-20 05:30:00,05h 35m,2024-06-20 11:05:00,RTC Bus Stand (Sindhu Travels),0 days 05:35:00
2,2,NueGo,314.0,3.7,44.0,Electric A/C Seater (2+2),2024-06-20 07:00:00,06h 10m,2024-06-20 13:10:00,RTC Bus Stand (Sindhu Travels),0 days 06:10:00
3,3,AdRajesh Transports,1080.0,3.9,36.0,A/C Sleeper (2+1),2024-06-20 23:36:00,06h 24m,2024-06-20 06:00:00,RTC Bus Stand (Sindhu Travels),-1 days +06:24:00
4,4,Yolo Bus,299.0,4.9,37.0,Bharat Benz A/C Seater /Sleeper (2+1),2024-06-20 07:15:00,05h 58m,2024-06-20 13:13:00,RTC Bus Stand(NEW DOT TRAVELS),0 days 05:58:00


In [108]:
df['Travel_time'] = df['Travel_time'].dt.total_seconds()

In [109]:
df['Travel_time'] = df['Travel_time']/60
df.head()

,Unnamed: 0,Names,Bus_fare,Ratings,Seats,Bus_types,Starting_time,Duration,Ending_time,Starting_point,Travel_time
0,0,NueGo,314.0,4.0,33.0,Electric A/C Seater (2+2),2024-06-20 05:00:00,06h 10m,2024-06-20 11:10:00,TIRUPATHI (12:30Am),370.0
1,1,FRESHBUS,489.0,4.6,39.0,Electric A/C Seater (2+2),2024-06-20 05:30:00,05h 35m,2024-06-20 11:05:00,RTC Bus Stand (Sindhu Travels),335.0
2,2,NueGo,314.0,3.7,44.0,Electric A/C Seater (2+2),2024-06-20 07:00:00,06h 10m,2024-06-20 13:10:00,RTC Bus Stand (Sindhu Travels),370.0
3,3,AdRajesh Transports,1080.0,3.9,36.0,A/C Sleeper (2+1),2024-06-20 23:36:00,06h 24m,2024-06-20 06:00:00,RTC Bus Stand (Sindhu Travels),-1056.0
4,4,Yolo Bus,299.0,4.9,37.0,Bharat Benz A/C Seater /Sleeper (2+1),2024-06-20 07:15:00,05h 58m,2024-06-20 13:13:00,RTC Bus Stand(NEW DOT TRAVELS),358.0


In [110]:
df[['hours','minutes']] = df.Duration.str.split(' ', expand = True)

In [122]:
df['hours'] = df['hours'].str.replace('h','').replace('0-16','16').replace('0-18','18').replace('0-14','14').astype(float)
df['minutes'] = df['minutes'].str.replace('m','').replace('0-35','35').replace('0-18','18').replace('0-14','14').astype(float)

In [123]:

df['hours'] = df['hours']*60
df['minutes'] = df['minutes'].str.replace('m','').astype(float)
df['Total_time'] = df['hours'] + df['minutes']
df.head()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Hp\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Hp\AppData\Local\Temp\ipykernel_9688\1836067432.py", line 2, in <module>
    df['minutes'] = df['minutes'].str.replace('m','').astype(float)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Hp\anaconda3\Lib\site-packages\pandas\core\generic.py", line 6324, in astype
    try:
         
  File "C:\Users\Hp\anaconda3\Lib\site-packages\pandas\core\internals\managers.py", line 451, in astype
    copy = False
       ^^^^^^^^^
  File "C:\Users\Hp\anaconda3\Lib\site-packages\pandas\core\internals\managers.py", line 352, in apply
    if obj.ndim == 1:
      ^^^^^^^^^^^^^^^^
  File "C:\Users\Hp\anaconda3\Lib\site-packages\pandas\core\internals\blocks.py", line 511, in astype
    and is_float_dtype(other.dtype)
             ^^^^^^^^^^^^^^^^^^^^^^^
  Fil

In [114]:
df.Duration.info()

<class 'pandas.core.series.Series'>
RangeIndex: 3931 entries, 0 to 3930
Series name: Duration
Non-Null Count  Dtype 
--------------  ----- 
3931 non-null   object
dtypes: object(1)
memory usage: 30.8+ KB


In [ ]:
df